In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import tensorflow as tf
display(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [6]:
# train = pd.read_csv('data/train.csv')
# test = pd.read_csv('data/test.csv')

In [7]:
from sklearn.model_selection import train_test_split
# y_train = train.label
# X_train = train.loc[:, train.columns != 'label']

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
num_classes = 10
# epochs = 20

In [9]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
train_data = np.array(train_df, dtype = 'float32')
test_data = np.array(test_df, dtype='float32')

In [10]:
train_data.shape

(42000, 785)

In [11]:
x_train = train_data[:,1:]/255
y_train = train_data[:,0]
x_test= test_data[:]/255

x_train,x_validate,y_train,y_validate = train_test_split(x_train,y_train,test_size = 0.2,random_state = 12345)

In [14]:
image_rows = 28
image_cols = 28
batch_size = 4096
image_shape = (image_rows,image_cols,1) 

In [15]:
print(x_train.shape)
print(x_test.shape)

(33600, 784)
(28000, 784)


In [29]:
x_train = x_train.reshape(-1,*image_shape)
x_train.shape

(33600, 28, 28, 1)

In [19]:
x_train = x_train.reshape(-1,*image_shape)
x_test = x_test.reshape(-1,*image_shape)
x_validate = x_validate.reshape(-1,*image_shape)

In [54]:
cnn_model_al = Sequential([
    # CONVOLUTIONAL PART
    Conv2D(filters=16,kernel_size=3,activation='relu',input_shape = image_shape),
    MaxPooling2D(pool_size=2) ,# down sampling the output instead of 28*28 it is 14*14
    Dropout(0.2),
    
    #LINEAR PART
    Flatten(), # flatten out the layers
    Dense(32,activation='relu'),
    Dense(10,activation = 'softmax')
    
])

cnn_model_al.compile(loss ='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001),metrics =['accuracy'])

In [47]:

cnn_model_small = Sequential([
    Conv2D(filters=16,kernel_size=3,activation='relu',input_shape = image_shape),
    MaxPooling2D(pool_size=2) ,# down sampling the output instead of 28*28 it is 14*14
    Dropout(0.2),
    
    Conv2D(filters=32,kernel_size=3,activation='relu',input_shape = image_shape),
    MaxPooling2D(pool_size=2) ,# down sampling the output instead of 14 -> 7
    Dropout(0.3),
    
    Conv2D(filters=64,kernel_size=3,activation='relu',input_shape = image_shape),
    MaxPooling2D(pool_size=2) ,# down sampling the output instead of 7 -> 3 or 4
    Dropout(0.4),
    
    Flatten(), # flatten out the layers
    

    Dense(128,activation='relu'),
    Dense(64,activation='relu'),
    Dense(32,activation='relu'),
    Dense(10,activation = 'softmax')
    
])

cnn_model_small.compile(loss ='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001),metrics =['accuracy'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [45]:
with tf.device('/CPU:0'):
    history = cnn_model_al.fit(
        x_train,
        y_train,
        batch_size=100000,
        epochs=10,
        verbose=1,
        validation_data=(x_validate,y_validate),
    )

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.0160 - accuracy: 0.9948 - val_loss: 0.0591 - val_accuracy: 0.9843
Epoch 2/10
1/1 [==============================] - 3s 3s/step - loss: 0.0164 - accuracy: 0.9949 - val_loss: 0.0590 - val_accuracy: 0.9843
Epoch 3/10
1/1 [==============================] - 3s 3s/step - loss: 0.0162 - accuracy: 0.9949 - val_loss: 0.0589 - val_accuracy: 0.9844
Epoch 4/10
1/1 [==============================] - 3s 3s/step - loss: 0.0159 - accuracy: 0.9952 - val_loss: 0.0588 - val_accuracy: 0.9843
Epoch 5/10
1/1 [==============================] - 3s 3s/step - loss: 0.0167 - accuracy: 0.9945 - val_loss: 0.0588 - val_accuracy: 0.9843
Epoch 6/10
1/1 [==============================] - 3s 3s/step - loss: 0.0157 - accuracy: 0.9949 - val_loss: 0.0587 - val_accuracy: 0.9843
Epoch 7/10
1/1 [==============================] - 3s 3s/step - loss: 0.0158 - accuracy: 0.9952 - val_loss: 0.0586 - val_accuracy: 0.9843
Epoch 8/10
1/1 [=========================

In [55]:
with tf.device('/GPU:0'):
    history = cnn_model_al.fit(
        x_train,
        y_train,
        batch_size=4000,
        epochs=10,
        verbose=1,
        validation_data=(x_validate,y_validate),
    )

Epoch 1/10
7/9 [======================>.......] - ETA: 0s - loss: 2.1407 - accuracy: 0.3642

2023-05-12 19:51:21.945489: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:1014] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_7/dropout_11/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


9/9 [==============================] - 0s 24ms/step - loss: 2.0955 - accuracy: 0.3946 - val_loss: 1.7373 - val_accuracy: 0.5925
Epoch 2/10
9/9 [==============================] - 0s 11ms/step - loss: 1.4994 - accuracy: 0.6452 - val_loss: 1.1556 - val_accuracy: 0.7400
Epoch 3/10
9/9 [==============================] - 0s 12ms/step - loss: 0.9745 - accuracy: 0.7701 - val_loss: 0.7463 - val_accuracy: 0.8164
Epoch 4/10
9/9 [==============================] - 0s 12ms/step - loss: 0.6481 - accuracy: 0.8321 - val_loss: 0.5284 - val_accuracy: 0.8581
Epoch 5/10
9/9 [==============================] - 0s 12ms/step - loss: 0.4818 - accuracy: 0.8688 - val_loss: 0.4164 - val_accuracy: 0.8864
Epoch 6/10
9/9 [==============================] - 0s 12ms/step - loss: 0.3945 - accuracy: 0.8905 - val_loss: 0.3545 - val_accuracy: 0.9023
Epoch 7/10
9/9 [==============================] - 0s 12ms/step - loss: 0.3444 - accuracy: 0.9017 - val_loss: 0.3153 - val_accuracy: 0.9108
Epoch 8/10
9/9 [======================

In [49]:
with tf.device('/GPU:0'):
    history = cnn_model_small.fit(
        x_train,
        y_train,
        batch_size=4000,
        epochs=100,
        verbose=1,
        validation_data=(x_validate,y_validate),
    )

Epoch 1/100
9/9 [==============================] - 0s 20ms/step - loss: 0.5520 - accuracy: 0.8274 - val_loss: 0.3116 - val_accuracy: 0.9118
Epoch 2/100
9/9 [==============================] - 0s 15ms/step - loss: 0.5124 - accuracy: 0.8399 - val_loss: 0.2804 - val_accuracy: 0.9199
Epoch 3/100
9/9 [==============================] - 0s 15ms/step - loss: 0.4745 - accuracy: 0.8504 - val_loss: 0.2522 - val_accuracy: 0.9251
Epoch 4/100
9/9 [==============================] - 0s 15ms/step - loss: 0.4468 - accuracy: 0.8606 - val_loss: 0.2329 - val_accuracy: 0.9305
Epoch 5/100
9/9 [==============================] - 0s 16ms/step - loss: 0.4239 - accuracy: 0.8686 - val_loss: 0.2210 - val_accuracy: 0.9326
Epoch 6/100
9/9 [==============================] - 0s 15ms/step - loss: 0.3976 - accuracy: 0.8782 - val_loss: 0.2076 - val_accuracy: 0.9368
Epoch 7/100
9/9 [==============================] - 0s 15ms/step - loss: 0.3768 - accuracy: 0.8845 - val_loss: 0.1964 - val_accuracy: 0.9386
Epoch 8/100
9/9 [===

In [87]:

cnn_model_small = Sequential([
    Conv2D(filters=5,kernel_size=3,activation='relu',input_shape = image_shape),
    MaxPooling2D(pool_size=2) ,# down sampling the output instead of 28*28 it is 14*14
    Dropout(0.2),
    Flatten(), # flatten out the layers
    Dense(32,activation='relu'),
    Dense(10,activation = 'softmax')
    
])

cnn_model_small.compile(loss ='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001),metrics =['accuracy'])

Epoch 1/10
32/34 [===========================>..] - ETA: 0s - loss: 2.1665 - accuracy: 0.1936

2023-05-12 07:30:02.249924: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:1014] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_17/dropout_17/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


34/34 [==============================] - 1s 8ms/step - loss: 2.1545 - accuracy: 0.1975 - val_loss: 1.7967 - val_accuracy: 0.3451
Epoch 2/10
34/34 [==============================] - 0s 4ms/step - loss: 1.6719 - accuracy: 0.3943 - val_loss: 1.2051 - val_accuracy: 0.6589
Epoch 3/10
34/34 [==============================] - 0s 4ms/step - loss: 1.2328 - accuracy: 0.5714 - val_loss: 0.7506 - val_accuracy: 0.7854
Epoch 4/10
34/34 [==============================] - 0s 5ms/step - loss: 0.9746 - accuracy: 0.6643 - val_loss: 0.5724 - val_accuracy: 0.8351
Epoch 5/10
34/34 [==============================] - 0s 4ms/step - loss: 0.8325 - accuracy: 0.7187 - val_loss: 0.4607 - val_accuracy: 0.8785
Epoch 6/10
34/34 [==============================] - 0s 4ms/step - loss: 0.7422 - accuracy: 0.7542 - val_loss: 0.3967 - val_accuracy: 0.8973
Epoch 7/10
34/34 [==============================] - 0s 4ms/step - loss: 0.6786 - accuracy: 0.7775 - val_loss: 0.3384 - val_accuracy: 0.9127
Epoch 8/10
34/34 [=============

In [85]:
with tf.device('/GPU:0'):
    history = cnn_model_big.fit(
        x_train,
        y_train,
        batch_size=1000,
        epochs=10,
        verbose=1,
        validation_data=(x_validate,y_validate),
    )

Epoch 1/10
34/34 [==============================] - 0s 7ms/step - loss: 0.0895 - accuracy: 0.9739 - val_loss: 0.0970 - val_accuracy: 0.9721
Epoch 2/10
34/34 [==============================] - 0s 5ms/step - loss: 0.0807 - accuracy: 0.9769 - val_loss: 0.0929 - val_accuracy: 0.9713
Epoch 3/10
34/34 [==============================] - 0s 6ms/step - loss: 0.0776 - accuracy: 0.9769 - val_loss: 0.0880 - val_accuracy: 0.9742
Epoch 4/10
34/34 [==============================] - 0s 6ms/step - loss: 0.0709 - accuracy: 0.9794 - val_loss: 0.0840 - val_accuracy: 0.9751
Epoch 5/10
34/34 [==============================] - 0s 5ms/step - loss: 0.0667 - accuracy: 0.9810 - val_loss: 0.0821 - val_accuracy: 0.9756
Epoch 6/10
34/34 [==============================] - 0s 5ms/step - loss: 0.0629 - accuracy: 0.9814 - val_loss: 0.0768 - val_accuracy: 0.9777
Epoch 7/10
34/34 [==============================] - 0s 5ms/step - loss: 0.0602 - accuracy: 0.9823 - val_loss: 0.0776 - val_accuracy: 0.9767
Epoch 8/10
34/34 [==

In [59]:
with tf.device('/CPU:0'):
    history = cnn_model.fit(
        x_train,
        y_train,
        batch_size=2048,
        epochs=10,
        verbose=1,
        validation_data=(x_validate,y_validate),
    )

Epoch 1/10
17/17 [==============================] - 1s 34ms/step - loss: 2.2997 - accuracy: 0.1242 - val_loss: 2.2937 - val_accuracy: 0.1223
Epoch 2/10
17/17 [==============================] - 0s 28ms/step - loss: 2.2837 - accuracy: 0.1400 - val_loss: 2.2633 - val_accuracy: 0.1625
Epoch 3/10
17/17 [==============================] - 0s 28ms/step - loss: 2.2357 - accuracy: 0.2345 - val_loss: 2.1864 - val_accuracy: 0.3746
Epoch 4/10
17/17 [==============================] - 0s 28ms/step - loss: 2.1338 - accuracy: 0.3871 - val_loss: 2.0454 - val_accuracy: 0.5196
Epoch 5/10
17/17 [==============================] - 0s 27ms/step - loss: 1.9675 - accuracy: 0.5100 - val_loss: 1.8371 - val_accuracy: 0.6057
Epoch 6/10
17/17 [==============================] - 0s 27ms/step - loss: 1.7396 - accuracy: 0.5922 - val_loss: 1.5804 - val_accuracy: 0.6869
Epoch 7/10
17/17 [==============================] - 0s 27ms/step - loss: 1.4911 - accuracy: 0.6518 - val_loss: 1.3268 - val_accuracy: 0.7260
Epoch 8/10
17

In [46]:
with tf.device('/GPU:0'):
    history = cnn_model.fit(
        x_train,
        y_train,
        batch_size=1000,
        epochs=10,
        verbose=1,
        validation_data=(x_validate,y_validate),
    )

Epoch 1/10
34/34 [==============================] - 0s 6ms/step - loss: 6.2106e-04 - accuracy: 1.0000 - val_loss: 0.0755 - val_accuracy: 0.9851
Epoch 2/10
34/34 [==============================] - 0s 5ms/step - loss: 9.5650e-04 - accuracy: 0.9997 - val_loss: 0.0777 - val_accuracy: 0.9848
Epoch 3/10
34/34 [==============================] - 0s 5ms/step - loss: 0.0012 - accuracy: 0.9997 - val_loss: 0.0768 - val_accuracy: 0.9851
Epoch 4/10
34/34 [==============================] - 0s 5ms/step - loss: 9.5860e-04 - accuracy: 0.9997 - val_loss: 0.0770 - val_accuracy: 0.9851
Epoch 5/10
34/34 [==============================] - 0s 5ms/step - loss: 8.3605e-04 - accuracy: 0.9998 - val_loss: 0.0766 - val_accuracy: 0.9850
Epoch 6/10
34/34 [==============================] - 0s 5ms/step - loss: 7.4790e-04 - accuracy: 0.9998 - val_loss: 0.0775 - val_accuracy: 0.9851
Epoch 7/10
34/34 [==============================] - 0s 5ms/step - loss: 0.0012 - accuracy: 0.9996 - val_loss: 0.0782 - val_accuracy: 0.9849


In [55]:
with tf.device('/GPU:0'):
    history = cnn_model.fit(
        x_train,
        y_train,
        batch_size=20000,
        epochs=10,
        verbose=1,
        validation_data=(x_validate,y_validate),
    )

Epoch 1/10
2/2 [==============================] - 0s 113ms/step - loss: 6.4763e-04 - accuracy: 0.9999 - val_loss: 0.0781 - val_accuracy: 0.9852
Epoch 2/10
2/2 [==============================] - 0s 77ms/step - loss: 6.1660e-04 - accuracy: 0.9999 - val_loss: 0.0782 - val_accuracy: 0.9851
Epoch 3/10
2/2 [==============================] - 0s 79ms/step - loss: 6.5709e-04 - accuracy: 0.9999 - val_loss: 0.0782 - val_accuracy: 0.9851
Epoch 4/10
2/2 [==============================] - 0s 76ms/step - loss: 7.4968e-04 - accuracy: 0.9999 - val_loss: 0.0783 - val_accuracy: 0.9850
Epoch 5/10
2/2 [==============================] - 0s 75ms/step - loss: 5.7256e-04 - accuracy: 0.9999 - val_loss: 0.0783 - val_accuracy: 0.9850
Epoch 6/10
2/2 [==============================] - 0s 75ms/step - loss: 0.0010 - accuracy: 0.9997 - val_loss: 0.0783 - val_accuracy: 0.9850
Epoch 7/10
2/2 [==============================] - 0s 75ms/step - loss: 7.5087e-04 - accuracy: 0.9998 - val_loss: 0.0782 - val_accuracy: 0.9850
Ep

In [ ]:
import matplotlib.pyplot as plt

class_names = [str(i) for i in range(10)]
plt.figure(figsize=(10, 10))
for i in range(36):
    plt.subplot(6, 6, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i].reshape((28,28)))
    label_index = int(y_train[i])
    plt.title(class_names[label_index])
plt.show()

In [ ]:
history.history.keys()


In [ ]:
plt.figure(figsize=(10, 10))

plt.subplot(2, 2, 1)
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Training - Loss Function')

plt.subplot(2, 2, 2)
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Train - Accuracy')

In [ ]:
cnn_array = cnn_model.predict(x_test)

In [ ]:
cnn_label_array = [np.argmax(cnn_array[i]) for i in range(len(cnn_array))]

In [ ]:
cnn_output_df = pd.DataFrame()
cnn_output_df['Label'] = cnn_label_array
cnn_output_df.index +=1
cnn_output_df.index.name = "ImageId"
cnn_output_df.to_csv('/kaggle/working/submission.csv')